In [1]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg")

In [2]:

def add_padding(input, padding):
    rows = input.shape[0]
    columns = input.shape[1]
    output = np.zeros((rows + padding * 2, columns + padding * 2), dtype=float)
    output[ padding : rows + padding, padding : columns + padding] = input
    return output

def search_bounds(column, block_size, width, rshift):
    disparity_range = 250
    padding = block_size // 2
    right_bound = column
    if rshift:
        left_bound = column - disparity_range
        if left_bound < padding:
            left_bound = padding
        step = 1
    else:
        left_bound = column + disparity_range
        if left_bound >= (width - 2*padding):
            left_bound = width - 2*padding - 2
        step = -1
    return left_bound, right_bound, step

In [3]:
grayscale_max = 255

In [4]:
l = cv2.imread('D:/Disparity-Map-Generator/StereoMatchingTestings/Dolls/view1.png', cv2.IMREAD_GRAYSCALE)
r = cv2.imread('D:/Disparity-Map-Generator/StereoMatchingTestings/Dolls/view5.png', cv2.IMREAD_GRAYSCALE)

In [5]:

block_size = 5

In [6]:

padding = block_size // 2
left_img = add_padding(l, padding)
right_img = add_padding(r, padding)

In [7]:
height, width = left_img.shape

In [8]:
d_map = np.zeros(l.shape, dtype=float)

In [9]:
rshift = True

In [ ]:
for row in range(height - block_size + 1):
    for col in range(width - block_size + 1):

        bestdist = -1
        shift = 0
        left_pixel = left_img[row:row + block_size, col:col + block_size]
        l_bound, r_bound, step = search_bounds(col, block_size, width, rshift)

        for i in range(l_bound, r_bound, step):
            right_pixel = right_img[row:row + block_size, i:i + block_size]

            ssd = np.sum((left_pixel - right_pixel) ** 2)

            if ssd < bestdist:
                bestdist = ssd
                shift = i

            # ncc = np.sum(left_pixel*right_pixel)/((np.sum(left_pixel**2)*np.sum(right_pixel**2))**0.5)

            # if ncc/ssd > bestdist:
            #     bestdist = ncc/ssd
            #     shift = i

        if rshift:
            d_map[row, col] = col - shift
        else:
            d_map[row, col] = shift - col

In [ ]:
max_val = d_map.max()
# image = np.absolute(image)
image = np.divide(d_map, max_val)
disparity_SGBM = cv2.normalize(image*grayscale_max, image*grayscale_max, alpha=255,
                              beta=0, norm_type=cv2.NORM_MINMAX)
disparity_SGBM = np.uint8(disparity_SGBM)
cv2.imwrite('D:/Disparity-Map-Generator/PSNR_Assignment2/3.0 output/Dolls/disp1.png', disparity_SGBM)